In [1]:
import seaborn

%load_ext autoreload
%pylab
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
from pylab import rcParams
rcParams['figure.figsize'] = 11, 4
seaborn.set_style("whitegrid")
seaborn.despine()

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from mppfnet.mp_network import MPNetwork
from mppfnet.mp_problem import MPProblem
import gridopt
import pfnet

In [3]:
mp = MPNetwork()
mp.load('./data/case32.art')

In [4]:
mp.set_prices()

p = MPProblem(mp)
p.add_function(pfnet.FUNC_TYPE_NETCON_COST, 1.0)
p.add_constraint(pfnet.CONSTR_TYPE_DCPF)  # power flow
p.add_constraint(pfnet.CONSTR_TYPE_PAR_GEN_P)  # generator participation
p.add_constraint(pfnet.CONSTR_TYPE_PAR_GEN_Q)  # generator participation
p.analyze()
x = p.get_init_point()
p.eval(x)
p.construct_problem()

In [9]:
import scipy
network_a = scipy.sparse.block_diag([p.problem[i].A for i in range(mp.timesteps)])
network_b = np.hstack([p.problem[i].b for i in range(mp.timesteps)])
print(network_a.shape)

(271560, 271560)


In [14]:
print(network_b.shape)
(battery_a, battery_b) = p.construct_battery_constraint_matrices()
print(battery_b.shape)
b = np.hstack([network_b, battery_b])
print(b.shape)



(271560,)
(26280,)
(297840,)


In [5]:
import numpy as np
from optalg.opt_solver import OptSolverIQP, QuadProblem, OptSolverLCCP
problem = QuadProblem(p.Hphi,p.gphi,p.A,p.b,p.l,p.u)
solver = OptSolverIQP()
solver.set_parameters({'quiet': False, 'tol': 1e-7})
problem.show()

In [ ]:
solver.solve(problem)
print(solver.get_status())
problem.x


Solver: IQP
-----------
iter    phi      fmax      gmax       cu       cl       s    
 0 

In [ ]:
mp.generate_solar_profiles()
plot(mp.solar_profile_map[0])

In [ ]:
mp.generate_load_profiles()

In [ ]:
mp.get_load(1, 1).P

In [ ]:
plot(mp.load_profile_map[0])